In [ ]:
import numpy as np
import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [47]:
import numpy as np

In [ ]:
class VoxelMap:
    def __init__(self) -> None:
        pass


In [ ]:
class Graph:
    def __init__(self) -> None:
        pass


In [ ]:
class AlgorithmBase:
    def __init__(self) -> None:
        pass


In [46]:
# ppAlgorithm = DescreteAlgorithm()

# startState = np.array([0.0, 0.0, 0.0], dtype=float)
# goalState = np.array([8.0, 0.0, 8.0], dtype=float)
# ppAlgorithm.SetStartState(startState)
# ppAlgorithm.SetGoalState(goalState)

# ppAlgorithm.LoadVoxelMap(full_path="/home/ildar/catkin_ws/src/VRPP_ROS/launch/map.txt")